In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# 1.x Save Video

In [148]:
cap = cv2.VideoCapture(0)

#비디오 속성 확인
height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS) #초당 프레임수

#비디오 레코드 저장
videoWriter= cv2.VideoWriter('train4.avi',cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()
    
    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
        
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF ==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

# 2. Capture Landmarks & Export to CSV

In [149]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [150]:
landmarks=['shol']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [151]:
# 데이터 저장할 csv 파일 형성
#with open('coords4.csv',mode='w',newline='') as f:
#    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#    csv_writer.writerow(landmarks)

In [152]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        
        if action=='good':
            keypoints = np.insert(keypoints,0,0)

        if action=='left':
            keypoints = np.insert(keypoints,0,1)
            
        if action=='right':
            keypoints = np.insert(keypoints,0,2)
            
        if action=='fore':
            keypoints = np.insert(keypoints,0,3)
            
        if action=='back':
            keypoints = np.insert(keypoints,0,4)
            
        with open('coords4.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        pass

In [153]:
# 영상을 통해 데이터셋 구축
cap =cv2.VideoCapture("train4.avi") #setting video capture device(number은 웹캠을 대표하는 숫자)

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret,frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))
        
        #키보드로 입력한 키와 동일한 아스키코드값
        k = cv2.waitKey(1)
        
        if k == 103: # good의 g
            print('good')
            export_landmark(results,'good')
            
        if k == 108: # left의 l
            print('left')
            export_landmark(results,'left')
            
        if k == 114: # right의 r
            print('right')
            export_landmark(results,'right')
            
        if k == 102: # fore의 f
            print('fore')
            export_landmark(results,'fore')
            
        if k == 98: # back의 b
            print('back')
            export_landmark(results,'back')
            

        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.

good
left
left
right
right
good
left
left
good
right
good
good
left
left
good
right
right
right
good
good
fore
fore
good
good
back
back
back
left
left
good
good
right
right
right
good
good
left
left
left
good
good
right
right
right
right
good
fore
fore
fore
fore
fore
good
back
back
good
fore
fore
good
good
back
back
good


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# 3. Train Custom Model Using Scikit Learn

In [154]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [155]:
df = pd.read_csv('coords4.csv')

In [156]:
df[df['shol']==3] #왼쪽 치우침

,shol,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
53,3.0,0.560696,0.517402,-0.606031,0.999989,0.585385,0.438686,-0.567640,0.999970,0.599616,...,0.447624,0.000003,0.641757,3.293921,0.212254,0.000051,0.488603,3.280117,0.017631,0.000018
54,3.0,0.536820,0.585384,-0.755946,0.999984,0.570379,0.486834,-0.709715,0.999953,0.589985,...,0.340499,0.000001,0.596923,3.448244,0.039794,0.000024,0.443098,3.419838,-0.191399,0.000009
60,3.0,0.568007,0.508584,-0.567420,0.999981,0.588818,0.440438,-0.530896,0.999950,0.601933,...,0.391174,0.000009,0.596026,3.162393,0.141737,0.000072,0.448746,3.139543,-0.043415,0.000026
61,3.0,0.565814,0.527062,-0.645416,0.999987,0.589423,0.449453,-0.605132,0.999966,0.602694,...,0.683632,0.000003,0.618371,3.268411,0.324067,0.000040,0.467711,3.245472,0.231794,0.000014
62,3.0,0.554855,0.538278,-0.598904,0.999985,0.585771,0.449981,-0.569461,0.999952,0.601510,...,0.541527,0.000001,0.611038,3.398893,0.224944,0.000021,0.444086,3.371482,0.076471,0.000009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,3.0,0.560200,0.679096,-1.012036,0.999423,0.588663,0.601691,-0.951421,0.999476,0.605331,...,0.072118,0.000135,0.644270,3.458538,-0.315063,0.000425,0.467304,3.429666,-0.472404,0.000616
423,3.0,0.549045,0.650121,-1.080985,0.999392,0.574869,0.576813,-1.027721,0.999367,0.593031,...,0.429451,0.000090,0.630054,3.525110,-0.042588,0.000336,0.475078,3.500948,-0.186240,0.000440
424,3.0,0.545857,0.616342,-0.539944,0.999793,0.568857,0.552223,-0.501607,0.999760,0.585000,...,0.342182,0.000031,0.602483,3.283121,-0.022282,0.000149,0.446922,3.270551,-0.105604,0.000155
429,3.0,0.557531,0.711400,-1.044749,0.999405,0.584365,0.632507,-0.990530,0.999493,0.600332,...,-0.023861,0.000160,0.658870,3.312011,-0.366931,0.000435,0.497133,3.277951,-0.513559,0.000608


In [157]:
X = df.drop('shol',axis=1) #shol을 제외한 입력 feature만 남김.
y = df['shol']

In [158]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123) #어깨

In [159]:
y_test

13     2.0
319    1.0
142    2.0
201    1.0
291    3.0
      ... 
328    3.0
134    0.0
148    3.0
370    4.0
317    0.0
Name: shol, Length: 131, dtype: float64

https://camo.githubusercontent.com/37bc1d0f546f2dd001798a0f66057c9c907e6b5efae95a068bcce8c852c2e234/68747470733a2f2f692e696d6775722e636f6d2f336a38425064632e706e67

In [160]:
df.drop(['x13'],axis=1,inplace=True)
df.drop(['y13'],axis=1,inplace=True)
df.drop(['z13'],axis=1,inplace=True)
df.drop(['v13'],axis=1,inplace=True)

df.drop(['x14'],axis=1,inplace=True)
df.drop(['y14'],axis=1,inplace=True)
df.drop(['z14'],axis=1,inplace=True)
df.drop(['v14'],axis=1,inplace=True)

df.drop(['x15'],axis=1,inplace=True)
df.drop(['y15'],axis=1,inplace=True)
df.drop(['z15'],axis=1,inplace=True)
df.drop(['v15'],axis=1,inplace=True)

df.drop(['x16'],axis=1,inplace=True)
df.drop(['y16'],axis=1,inplace=True)
df.drop(['z16'],axis=1,inplace=True)
df.drop(['v16'],axis=1,inplace=True)

df.drop(['x17'],axis=1,inplace=True)
df.drop(['y17'],axis=1,inplace=True)
df.drop(['z17'],axis=1,inplace=True)
df.drop(['v17'],axis=1,inplace=True)

df.drop(['x18'],axis=1,inplace=True)
df.drop(['y18'],axis=1,inplace=True)
df.drop(['z18'],axis=1,inplace=True)
df.drop(['v18'],axis=1,inplace=True)

df.drop(['x19'],axis=1,inplace=True)
df.drop(['y19'],axis=1,inplace=True)
df.drop(['z19'],axis=1,inplace=True)
df.drop(['v19'],axis=1,inplace=True)

df.drop(['x20'],axis=1,inplace=True)
df.drop(['y20'],axis=1,inplace=True)
df.drop(['z20'],axis=1,inplace=True)
df.drop(['v20'],axis=1,inplace=True)

df.drop(['x21'],axis=1,inplace=True)
df.drop(['y21'],axis=1,inplace=True)
df.drop(['z21'],axis=1,inplace=True)
df.drop(['v21'],axis=1,inplace=True)

df.drop(['x22'],axis=1,inplace=True)
df.drop(['y22'],axis=1,inplace=True)
df.drop(['z22'],axis=1,inplace=True)
df.drop(['v22'],axis=1,inplace=True)

df.drop(['x23'],axis=1,inplace=True)
df.drop(['y23'],axis=1,inplace=True)
df.drop(['z23'],axis=1,inplace=True)
df.drop(['v23'],axis=1,inplace=True)

df.drop(['x24'],axis=1,inplace=True)
df.drop(['y24'],axis=1,inplace=True)
df.drop(['z24'],axis=1,inplace=True)
df.drop(['v24'],axis=1,inplace=True)

df.drop(['x25'],axis=1,inplace=True)
df.drop(['y25'],axis=1,inplace=True)
df.drop(['z25'],axis=1,inplace=True)
df.drop(['v25'],axis=1,inplace=True)

df.drop(['x26'],axis=1,inplace=True)
df.drop(['y26'],axis=1,inplace=True)
df.drop(['z26'],axis=1,inplace=True)
df.drop(['v26'],axis=1,inplace=True)

df.drop(['x27'],axis=1,inplace=True)
df.drop(['y27'],axis=1,inplace=True)
df.drop(['z27'],axis=1,inplace=True)
df.drop(['v27'],axis=1,inplace=True)

df.drop(['x28'],axis=1,inplace=True)
df.drop(['y28'],axis=1,inplace=True)
df.drop(['z28'],axis=1,inplace=True)
df.drop(['v28'],axis=1,inplace=True)

df.drop(['x29'],axis=1,inplace=True)
df.drop(['y29'],axis=1,inplace=True)
df.drop(['z29'],axis=1,inplace=True)
df.drop(['v29'],axis=1,inplace=True)

df.drop(['x30'],axis=1,inplace=True)
df.drop(['y30'],axis=1,inplace=True)
df.drop(['z30'],axis=1,inplace=True)
df.drop(['v30'],axis=1,inplace=True)

df.drop(['x31'],axis=1,inplace=True)
df.drop(['y31'],axis=1,inplace=True)
df.drop(['z31'],axis=1,inplace=True)
df.drop(['v31'],axis=1,inplace=True)

df.drop(['x32'],axis=1,inplace=True)
df.drop(['y32'],axis=1,inplace=True)
df.drop(['z32'],axis=1,inplace=True)
df.drop(['v32'],axis=1,inplace=True)

df.drop(['x33'],axis=1,inplace=True)
df.drop(['y33'],axis=1,inplace=True)
df.drop(['z33'],axis=1,inplace=True)
df.drop(['v33'],axis=1,inplace=True)

In [161]:
df

,shol,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z10,v10,x11,y11,z11,v11,x12,y12,z12,v12
0,0.0,0.543709,0.524400,-0.569407,0.999989,0.567922,0.459572,-0.536507,0.999967,0.581814,...,-0.484794,0.999982,0.514014,0.596661,-0.480938,0.999988,0.733565,0.901072,-0.189265,0.999300
1,0.0,0.558027,0.499163,-0.688157,0.999974,0.579239,0.433892,-0.642779,0.999927,0.591282,...,-0.578711,0.999954,0.529008,0.574013,-0.595632,0.999972,0.731214,0.863917,-0.159800,0.999147
2,0.0,0.568836,0.497314,-0.663163,0.999976,0.588036,0.432810,-0.618399,0.999941,0.601490,...,-0.561716,0.999967,0.542423,0.570753,-0.568092,0.999969,0.734886,0.873584,-0.181027,0.999395
3,0.0,0.572842,0.500612,-0.553825,0.999985,0.591226,0.436276,-0.517266,0.999964,0.604136,...,-0.464990,0.999977,0.543537,0.572848,-0.466101,0.999980,0.742385,0.862153,-0.157847,0.999482
4,0.0,0.536347,0.485976,-0.519811,0.999985,0.557511,0.431650,-0.486413,0.999955,0.572071,...,-0.438862,0.999967,0.510053,0.560505,-0.443975,0.999980,0.719222,0.862548,-0.172011,0.999234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,0.0,0.548865,0.657911,-0.828266,0.999699,0.572691,0.594254,-0.774898,0.999524,0.586732,...,-0.695009,0.999558,0.522010,0.732256,-0.703386,0.999539,0.727077,0.980747,-0.178120,0.989111
432,0.0,0.548715,0.658024,-0.791155,0.999785,0.571695,0.595932,-0.738959,0.999623,0.585198,...,-0.663181,0.999699,0.521976,0.729517,-0.671809,0.999725,0.718519,0.970728,-0.168830,0.992657
433,4.0,0.539148,0.710834,-0.616601,0.999803,0.559258,0.654591,-0.574264,0.999582,0.571156,...,-0.511501,0.999720,0.517316,0.775471,-0.517396,0.999780,0.675655,0.957882,-0.136855,0.996580
434,4.0,0.539972,0.709536,-0.594722,0.999867,0.557652,0.653605,-0.554860,0.999696,0.567680,...,-0.489737,0.999806,0.517026,0.774456,-0.502717,0.999850,0.678562,0.959702,-0.087339,0.997838


In [162]:
df.mean()

shol    1.635321
x1      0.549815
y1      0.604326
z1     -0.667562
v1      0.999841
x2      0.570238
y2      0.539782
z2     -0.625427
v2      0.999696
x3      0.583817
y3      0.541513
z3     -0.625416
v3      0.999756
x4      0.596352
y4      0.544182
z4     -0.625491
v4      0.999672
x5      0.528109
y5      0.539590
z5     -0.630756
v5      0.999711
x6      0.513647
y6      0.541016
z6     -0.630261
v6      0.999775
x7      0.500476
y7      0.543138
z7     -0.630532
v7      0.999724
x8      0.615648
y8      0.581606
z8     -0.349783
v8      0.999701
x9      0.483647
y9      0.579415
z9     -0.360568
v9      0.999800
x10     0.576179
y10     0.678744
z10    -0.562793
v10     0.999770
x11     0.524394
y11     0.678194
z11    -0.566209
v11     0.999803
x12     0.723479
y12     0.936779
z12    -0.178420
v12     0.995743
dtype: float64

In [163]:
for val in range(1,12+1):
    print(df['v{}'.format(val)].mean())

0.9998414022933453
0.99969619772303
0.9997560481686111
0.9996720805354075
0.9997105845891008
0.9997747008680203
0.999724373768229
0.9997008043691653
0.9997997352289497
0.9997700454171644
0.9998029250344005
0.9957429465897586


## 3.2 Train Machine Learning Classification Model

In [164]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [165]:
pipelines = {
    #'lr' : make_pipeline(StandardScaler(),LogisticRegression(max_iter=1300)),
    #'rc' : make_pipeline(StandardScaler(),RidgeClassifier(max_iter=1300)),
    'rf' : make_pipeline(StandardScaler(),RandomForestClassifier(max_features=2000)),
    #'gb' : make_pipeline(StandardScaler(),GradientBoostingClassifier(n_estimators=1300))
}

In [166]:
fit_models={}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [167]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier',
                  RandomForestClassifier(max_features=2000))])}

In [168]:
fit_models['rf'].predict(X_test)

array([2., 1., 2., 1., 3., 1., 0., 0., 1., 0., 2., 3., 0., 1., 2., 2., 4.,
       4., 2., 2., 0., 4., 0., 0., 2., 0., 1., 0., 4., 0., 1., 2., 1., 3.,
       4., 2., 1., 1., 2., 4., 2., 2., 3., 2., 4., 0., 4., 4., 3., 1., 0.,
       0., 0., 0., 4., 1., 3., 3., 0., 1., 4., 2., 2., 0., 1., 4., 2., 2.,
       0., 0., 0., 0., 3., 0., 0., 2., 1., 0., 0., 0., 4., 2., 3., 4., 0.,
       3., 1., 3., 1., 2., 3., 0., 0., 0., 2., 4., 4., 1., 0., 0., 1., 4.,
       3., 3., 2., 3., 0., 0., 0., 3., 1., 0., 2., 4., 0., 1., 0., 2., 0.,
       4., 1., 3., 4., 0., 0., 1., 3., 0., 3., 4., 0.])

## 3.3 Evaluate and Serialize Model

In [169]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [170]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,pos_label=1),
         recall_score(y_test.values,yhat,pos_label=1))

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
yhat = fit_models['rf'].predict(X_test)

In [ ]:
yhat[:10]

In [ ]:
y_test

In [171]:
with open('Shoulder.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4. Make Detections with Model

In [172]:
with open('Shoulder.pkl','rb') as f:
    model = pickle.load(f)

In [173]:
landmarks=['shol']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [174]:
#VISUALIZE DEGREE
cap =cv2.VideoCapture(0) #setting video capture device(number은 웹캠을 대표하는 숫자)

# Curl counter variables
counter = 0

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened(): #실시간 영상을 가져올 수 있도록 함.
        ret, frame = cap.read() #frame은 현재 프레임 이미지가 담긴 것.
      
        #Detect stuff and render
        #Recolor image to RGB
        image =cv2.cvtColor(frame,cv2.COLOR_BGR2RGB) #웹캠으로 읽어온 frame을 BGR에서 RGB로 변환(Mediapipe는 RGB 형식임.)
        image.flags.writeable =False #이미지를 불변으로 설정하여 처리 속도를 향상 시킴.
        
        #Make detection -> 자세 detection을 results라는 변수에
        results= pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True #image 위에 그릴 수 있도록.
        image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Mediapipe 처리 결과를 BGR로 변환
        
         #Extract landmarks
        try:  
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
            X = pd.DataFrame([row],columns=landmarks[1:])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 0 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'Good'
            elif current_stage == 'Good' and body_language_class ==1 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'left'
                counter +=1
            elif current_stage == 'Good' and body_language_class ==2 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'right'
                counter +=1
            elif current_stage == 'Good' and body_language_class ==3 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'fore'
                counter +=1
            elif current_stage == 'Good' and body_language_class ==4 and body_language_prob[body_language_prob.argmax()] >= .7:
                current_stage = 'back'
                counter +=1
        
            #Setup status bow
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1) 


            #지금 상태
            cv2.putText(image,'CLASS',(15,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image, 'left' if body_language_class == 1 
                        else 'right' if body_language_class == 2 
                        else 'fore' if body_language_class == 3 
                        else 'back' if body_language_class == 4
                        else 'Good',(15,40)
                        ,cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            #Stage data 
            cv2.putText(image,'Count',(180,12),
                       cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
    
            cv2.putText(image,str(counter),(175,40),
                       cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            # Render detections
            #drawing 툴을 사용해서 감지된 자세포인트와 이들 간의 포인트를 연결해 보여준다.
            #mp_drawing.DrawingSpec은 관절부위와 bone 부분의 색깔, 굵기 등을 지정해준다.
            mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                     mp_drawing.DrawingSpec(color=(155,117,166),thickness=2,circle_radius=2),
                                     mp_drawing.DrawingSpec(color=(195,116,230),thickness=2,circle_radius=2))


        except: #error가 있으면 실행x
            pass
        
        cv2.imshow('Mediapipe Feed', image) #웹캠에서의 실시간 영상 확인 가능

        if cv2.waitKey(10) & 0xFF == ord('q'): #웹캠 화면을 종료하는 방법
            break

    cap.release() #비디오 객체 해제
    cv2.destroyAllWindows() #열린 opencv 창 전부 닫음.